In [30]:
from matplotlib import pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

plt.style.use('Solarize_Light2')

- **Load PCEPI data**

In [31]:
PCEPI_df = pd.read_csv('../data/PCEPI.csv')

PCEPI_df['DATE'] = pd.to_datetime(PCEPI_df['DATE'])
PCEPI_df.set_index('DATE', inplace=True)

PCEPI_df['^ %'] = PCEPI_df['PCEPI'].pct_change()

PCEPI_df.tail()

,PCEPI,^ %
DATE,,
2025-05-01,126.380,0.001823
2025-06-01,126.743,0.002872
2025-07-01,126.957,0.001688
2025-08-01,127.287,0.002599
2025-09-01,127.626,0.002663


- **Average change**

In [32]:
print(f"Average change is {PCEPI_df['^ %'].mean() * 100}%")

Average change is 0.266928672212724%


- **Overview on change**

In [33]:
positive_filter = PCEPI_df['^ %'] > 0
negative_filter = PCEPI_df['^ %'] < 0
zero_filter = PCEPI_df['^ %'] == 0

# count
total_items = len(PCEPI_df)

# average change
avg_change = [
    PCEPI_df[positive_filter]['^ %'].mean(),
    PCEPI_df[negative_filter]['^ %'].mean(),
    0,
    ""
]

# frequency
frequency = [
    PCEPI_df[positive_filter]['^ %'].count(),
    PCEPI_df[negative_filter]['^ %'].count(),
    PCEPI_df[zero_filter]['^ %'].count()
]
frequency.append(frequency[0]/frequency[1])

# frequency pct
frequency_pct = [100 * frequency[i]/total_items for i in range(3)]
frequency_pct.append("")

# Prob adj
prob_adj = [frequency_pct[i] * avg_change[i] for i in range(3)]
prob_adj.append("")

pd.DataFrame({
    "%": [x * 100 for x in avg_change],
    "Frequency": frequency,
    "Frequency %": frequency_pct,
    "Prob Adjust % Change": prob_adj,
}, index=["Av Pos", "Av Neg", "Zero", "Ratio P/N"])

,%,Frequency,Frequency %,Prob Adjust % Change
Av Pos,0.300494,737.000000,92.009988,0.276485
Av Neg,-0.132024,60.000000,7.490637,-0.009889
Zero,0,3.000000,0.374532,0.0
Ratio P/N,,12.283333,,


- **Stats**

In [34]:
from scipy.stats import describe

# Display the results

stats = describe(PCEPI_df['^ %'].dropna().tolist())
pd.DataFrame(
    {
        'value': [
            str(stats.nobs),
            stats.minmax[0] * 100,
            stats.minmax[1] * 100,
            stats.mean * 100,
            PCEPI_df['^ %'].median() * 100,
            PCEPI_df['^ %'].mode(dropna=True)[0] * 100,
            stats.variance,
            stats.skewness,
            stats.kurtosis
        ]
    },
    index=['nobs', 'Min %', 'Max %', 'Mean %', "Median %", "Mode %", 'Variance', 'Skewness', 'Kurtosis'],
)

,value
nobs,800
Min %,-1.180132
Max %,1.228482
Mean %,0.266929
Median %,0.228358
Mode %,0.0
Variance,0.000006
Skewness,0.510221
Kurtosis,2.971513


- **Data preview**

In [35]:
# Define bins
bins = [0.005 * i for i in range(-1, 3)]
bins.append(stats.minmax[1])
bins.insert(0, stats.minmax[0])
bin_labels = [f"{round(bins[i] * 100, 2)}% to {round(bins[i+1] * 100, 2)}%" for i in range(len(bins) - 1)]
bin_labels[0] = f"Less than {bins[1] * 100}%"
bin_labels[-1] = f"Greater than {bins[-2] * 100}%"

# Assign data to bins
binned = pd.cut(PCEPI_df['^ %'], bins=bins, labels=bin_labels, include_lowest=True)

# Calculate frequency, probability, and cumulative probability
frequency = binned.value_counts().sort_index()
probability = 100 * frequency / frequency.sum()
cumulative_probability = probability.cumsum()

occurrence_frequencies = pd.DataFrame({
    'Frequency': frequency.values,
    'Probability %': probability.values,
    'Cumulative Probability %': cumulative_probability.values
}, index=bin_labels)

occurrence_frequencies

,Frequency,Probability %,Cumulative Probability %
Less than -0.5%,3,0.375,0.375
-0.5% to 0.0%,60,7.500,7.875
0.0% to 0.5%,625,78.125,86.000
0.5% to 1.0%,101,12.625,98.625
Greater than 1.0%,11,1.375,100.000


In [36]:
fig = px.bar(occurrence_frequencies, x=occurrence_frequencies.index, y='Probability %')

fig = px.bar(
    occurrence_frequencies,
    x=occurrence_frequencies.index,
    y='Probability %',
)

fig.update_layout(
    title=dict(text='PCEPI % change'),
    width=800,
    height=600,
    yaxis=dict(title="Frequency"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()

- **PCEPI % change**

In [37]:
fig = go.Figure()

# Reference line
fig.add_trace(go.Scatter(
    x=PCEPI_df.index,
    y=[100 * stats.mean] * (stats.nobs + 1),
    name='Average % change',
    line=dict(
        width=2,
        color="#F2C14E",   # muted gold
        dash="dash"
    ),
    hoverinfo="skip",
    # showlegend=False,
))

# Bars
fig.add_trace(go.Bar(
    x=PCEPI_df.index,
    y=PCEPI_df['^ %'] * 100,
    name='PCEPI',
    marker=dict(
        color="#4C78A8",  # professional blue
        line=dict(color="#4C78A8", width=0)
    ),
    width=0.5,
))

fig.update_layout(
    title=dict(
        text='PCEPI % change',
        x=0.02,
        font=dict(size=20)
    ),
    plot_bgcolor="#0E1117",
    paper_bgcolor="#0E1117",
    font=dict(color="#E6E6E6", size=14),
    width=1400,
    height=700,
    yaxis=dict(
        title="%",
        exponentformat="none",
        showgrid=False,
        zeroline=False
    ),
    xaxis=dict(
        showgrid=False,
        zeroline=False
    ),
    hovermode="x unified",
)

fig.show()


- **WPSFD49207 data plot**

In [38]:
fig = px.line(
    PCEPI_df,
    x=PCEPI_df.index,
    y="PCEPI",
    markers=True
)

fig.update_layout(
    title=dict(text='Producer Price Index by Commodity: Final Demand: Finished Goods Less Food and Energy'),
    width=1000,
    height=600,
    yaxis=dict(title="PCEPI"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()